In [21]:
import os
import glob
import cv2
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np

from PIL import Image

In [22]:
def maxmin_norm(data):
    MAX = np.amax(data)
    MIN = np.amin(data)
    data = (data - MIN)/(MAX-MIN)
    return data

In [23]:
def SpotTheDifference_Generator(dataA, dataB, name_dataset, n_slice=1, name_tag=""):
    # shape supposed to be 512*512*284 by default
    assert dataA.shape == dataB.shape, ("DataA should share the same shape with DataB.")
    path2save = "./pytorch-CycleGAN-and-pix2pix/datasets/"+name_dataset+"/train/"
    h, w, c = dataA.shape
    img = np.zeros((n_slice, h, w*2))
        
    if n_slice == 1:
        for idx in range(c):
            img[:, :, :w] = dataA[:, :, idx]
            img[:, :, w:] = dataB[:, :, idx]
            name2save = path2save+name_tag+"_"+str(idx)+".npy"
            np.save(name2save, img)
        print(str(c)+" images have been saved.")
    else:
        for idx in range(c):
            if idx == 0:
                img[0, :, :w] = dataA[:, :, 0]
                img[1, :, :w] = dataA[:, :, 0]
                img[2, :, :w] = dataA[:, :, 1]
                
                img[0, :, w:] = dataB[:, :, 0]
                img[1, :, w:] = dataB[:, :, 0]
                img[2, :, w:] = dataB[:, :, 1]
            else:
                if idx == c-1:
                    img[0, :, :w] = dataA[:, :, c-2]
                    img[1, :, :w] = dataA[:, :, c-1]
                    img[2, :, :w] = dataA[:, :, c-1]
                    
                    img[0, :, w:] = dataB[:, :, c-2]
                    img[1, :, w:] = dataB[:, :, c-1]
                    img[2, :, w:] = dataB[:, :, c-1]
                else:
                    img[0, :, :w] = dataA[:, :, idx-1]
                    img[1, :, :w] = dataA[:, :, idx]
                    img[2, :, :w] = dataA[:, :, idx+1]
                    
                    img[0, :, w:] = dataB[:, :, idx-1]
                    img[1, :, w:] = dataB[:, :, idx]
                    img[2, :, w:] = dataB[:, :, idx+1]
            name2save = path2save+name_tag+"_"+str(idx)+".npy"
            np.save(name2save, img)
        print(str(c)+" images have been saved.")

In [24]:
def SingleImage_Generator(dataA, name_dataset, n_slice=1, name_tag=""):
    # shape supposed to be 512*512*284 by default
    path2save = "./pytorch-CycleGAN-and-pix2pix/datasets/"+name_dataset+"/test/"
    h, w, c = dataA.shape
    img = np.zeros((n_slice, h, w))
        
    if n_slice == 1:
        for idx in range(c):
            img[:, :, :] = dataA[:, :, idx]
            img = np.asarray(img, dtype=np.float())
            name2save = path2save+name_tag+"_"+str(idx)+".npy"
            np.save(name2save, img)
        print(str(c)+" images have been saved.")
    else:
        for idx in range(c):
            if idx == 0:
                img[0, :, :] = dataA[:, :, 0]
                img[1, :, :] = dataA[:, :, 0]
                img[2, :, :] = dataA[:, :, 1]
            else:
                if idx == c-1:
                    img[0, :, :] = dataA[:, :, c-2]
                    img[1, :, :] = dataA[:, :, c-1]
                    img[2, :, :] = dataA[:, :, c-1]
                else:
                    img[0, :, :] = dataA[:, :, idx-1]
                    img[1, :, :] = dataA[:, :, idx]
                    img[2, :, :] = dataA[:, :, idx+1]
            name2save = path2save+name_tag+"_"+str(idx)+".npy"
            img = np.asarray(img, dtype=np.float())
            np.save(name2save, img)
        print(str(c)+" images have been saved.")

In [28]:
name_dataset = "2d_enhanced_v4"
n_slice = 3

import os

for folder_name in ["train", "test"]:
    path = "./pytorch-CycleGAN-and-pix2pix/datasets/"+name_dataset+"/"+folder_name+"/"
    if not os.path.exists(path):
        os.makedirs(path)

## test dataset

In [55]:
list_ori = glob.glob("./data/"+name_dataset+"/test/*.nii")
list_ori.sort()
print("Test:")
for path_ori in list_ori:
    filename_ori = os.path.basename(path_ori)[:]
    filename_ori = filename_ori[:filename_ori.find(".")]
    print(filename_ori)
    data_ori = maxmin_norm(nib.load(path_ori).get_fdata())
    SingleImage_Generator(dataA=data_ori, name_dataset=name_dataset, n_slice=n_slice, name_tag=filename_ori)
    print("------------------------------------------------------------------------")

Test:
subj01_pet
284 images have been saved.
------------------------------------------------------------------------


## training dataset

In [29]:
list_ori = glob.glob("./data/"+name_dataset+"/pure/*.nii")
list_ori.sort()
for path_ori in list_ori:
    print("TrainA:")
    filename_ori = os.path.basename(path_ori)[:]
    filename_ori = filename_ori[:filename_ori.find(".")]
    print(filename_ori)
    data_ori = maxmin_norm(nib.load(path_ori).get_fdata())
    
    list_sim = glob.glob("./data/"+name_dataset+"/blur/*"+filename_ori+"*.nii")
    list_sim.sort()
    
    for path_sim in list_sim:
        print("Pairs")
        filename_sim = os.path.basename(path_sim)[:]
        filename_sim = filename_sim[:filename_sim.find(".")]
        print("A:", filename_ori)
        print("B:", filename_sim)
                
        data_sim = maxmin_norm(nib.load(path_sim).get_fdata())
        SpotTheDifference_Generator(dataA=data_ori, dataB=data_sim,
                                    name_dataset=name_dataset, n_slice=n_slice, name_tag=filename_sim)
        
    print("------------------------------------------------------------------------")
        
        
        
#         data_ori = nib.load(path_ori).get_fdata()
#         norm_ori = maxmin_norm(data_ori)*255
#         sliced_save(data=norm_ori,
#                     name_tag=os.path.basename(path_ori)[:-4],
#                     path2save="./pytorch-CycleGAN-and-pix2pix/datasets/"+name_dataset+"/"+folder_name+"/",
#                     n_slice=n_slice)

TrainA:
subj01_inv
Pairs
A: subj01_inv
B: fwhm_11_subj01_inv
284 images have been saved.
Pairs
A: subj01_inv
B: fwhm_13_subj01_inv
284 images have been saved.
Pairs
A: subj01_inv
B: fwhm_3_subj01_inv
284 images have been saved.
Pairs
A: subj01_inv
B: fwhm_5_subj01_inv
284 images have been saved.
Pairs
A: subj01_inv
B: fwhm_7_subj01_inv
284 images have been saved.
Pairs
A: subj01_inv
B: fwhm_9_subj01_inv
284 images have been saved.
------------------------------------------------------------------------


In [25]:
A = np.ones((3,3))
print(A)

[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]


In [26]:
B = np.ones((3,3))*8
print(B)

[[8. 8. 8.]
 [8. 8. 8.]
 [8. 8. 8.]]


In [27]:
C = np.zeros((6,3))
C[:3,:]=A
C[3:,:]=B
print(C)

[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]
 [8. 8. 8.]
 [8. 8. 8.]
 [8. 8. 8.]]
